# SecurityData Tests

This notebook explains the key functions of the SecurityData class that are used across the system. The SecurityData class is used to retrieve the Financial Statement and Metadata datasets. It stores and retrieves the data in formats ready for use in the LLM prompts.

In [17]:
import sys
import importlib
sys.path.insert(0, '..')

from requesters.company_data import SecurityData
from prompts import SYSTEM_PROMPTS

In [2]:
# Initialise the sec_helper object with the Dow Jones datasets downloaded in the 
# 01 Financial Data.ipynb file.
sec_helper = SecurityData('tmp/fs','dow_quarterly_ltm_v3.json')

In [30]:
# Find the total number of unique securities in the dataset
unique_securities = sec_helper.get_unique_securities()
len(unique_securities)

45

### Get all of the reporting dates
This is an important function as it is used to loop through the entire universe of reporting periods to run with the LLMs. 

In [32]:
all_reporting_dates = sec_helper.get_dates()
print(f'Total number of reporting dates: {len(all_reporting_dates)}')
all_reporting_dates[:5]

Total number of reporting dates: 449


['2020-01-08', '2020-01-14', '2020-01-15', '2020-01-23', '2020-01-24']

In [33]:
# View a list of securities that are reporting on the reporting date
sec_helper.get_securities_reporting_on_date('2020-01-23')

['INTC UQ Equity', 'PG UN Equity', 'INTC UW Equity']

### View all of the data for a security on a reporting date
This example shows that Procter and Gamble reported their Q4 2019 earning on 23rd January 2020. Their Q4 2019 period concluded on 31st December 2019, but the information was not released to the mar

In [15]:
data_for_PG = sec_helper.get_security_all_data('2020-01-23','PG UN Equity')
print(data_for_PG['name'])
print(data_for_PG['sector'])
print(data_for_PG['figi'])
print(data_for_PG['sec_fs'])
print(data_for_PG['stock_price'])

Procter & Gamble Co/The
Consumer Staples
BBG000BR2TH3
Income Statement:                                                        t           t-1           t-2           t-3           t-4           t-5
items                                                                                                                          
Revenue                                      6.959400e+10  6.879200e+10  6.768400e+10  6.709300e+10  6.691200e+10  6.686900e+10
Cost of Revenue                              3.495700e+10  3.500700e+10  3.476800e+10  3.485900e+10  3.485700e+10  3.464700e+10
Gross Profit                                 3.463700e+10  3.378500e+10  3.291600e+10  3.223400e+10  3.205500e+10  3.222200e+10
Operating Expenses                           2.782800e+10  2.756200e+10  2.742900e+10  1.887800e+10  1.880900e+10  1.895300e+10
Selling, General and Administrative Expense  1.948300e+10  1.921700e+10  1.908400e+10  1.887800e+10  1.880900e+10  1.895300e+10
Other Operating Expenses         

### Construct a prompt with a Security and Date

In [18]:
sec_helper.get_prompt('2020-01-08','WBA UW Equity', SYSTEM_PROMPTS['BASE']['prompt'])

[{'role': 'system',
  'content': "You are a financial analyst and must make a buy, sell or hold decision on a company based only on the provided datasets. Compute common financial ratios and then determine the buy or sell decision. Explain your reasons in less than 250 words. Provide a confidence score for how confident you are of the decision. If you are not confident then lower the confidence score. You must answer in a JSON format with a 'decision', 'confidence score' and 'reason'. Provide your answer in JSON format like the two examples: {'decision': BUY, 'confidence score': 80, 'reason': 'Gross profit and EPS have both increased over time'}, {'decision': SELL, 'confidence score': 90, 'reason': 'Price has declined and EPS is falling'} Company financial statements: {financials} "},
 {'role': 'user',
  'content': 'Income Statement:                                                        t           t-1           t-2           t-3           t-4           t-5\nitems                     

### Get a list of all of the companies/ dates to loop through

In [35]:
all_dates_securities = sec_helper.date_security_timeseries()
print(f'Total Number of reporting dates and securities: {len(all_dates_securities)}')
all_dates_securities[:10]

Total Number of reporting dates and securities: 952


[{'date': '2020-01-08', 'security': 'WBA UW Equity'},
 {'date': '2020-01-08', 'security': 'WBA UQ Equity'},
 {'date': '2020-01-14', 'security': 'JPM UN Equity'},
 {'date': '2020-01-15', 'security': 'UNH UN Equity'},
 {'date': '2020-01-23', 'security': 'INTC UQ Equity'},
 {'date': '2020-01-23', 'security': 'PG UN Equity'},
 {'date': '2020-01-23', 'security': 'INTC UW Equity'},
 {'date': '2020-01-24', 'security': 'AXP UN Equity'},
 {'date': '2020-01-28', 'security': 'PFE UN Equity'},
 {'date': '2020-01-28', 'security': 'RTX UN Equity'}]

### Finally, view the structure that the data is stored
We store the data in dictionary format in the following Hierarchy:

- date
     - company
       - Income Statement
       - Balance Sheet
       - Historical Stock Price
       - Metadata
         - Company Name
         - Sector
         - FIGI

In [24]:
all_data = sec_helper.get_all_data()
list(all_data.keys())[-10:]

['2025-03-20',
 '2025-04-08',
 '2025-04-11',
 '2025-04-14',
 '2025-04-16',
 '2025-04-17',
 '2025-04-18',
 '2025-04-22',
 '2025-04-23',
 '2025-04-24']

In [26]:
all_data['2025-03-20'].keys()

dict_keys(['HD UN Equity', 'NKE UN Equity'])

In [27]:
all_data['2025-03-20']['HD UN Equity'].keys()

dict_keys(['is', 'bs', 'px', 'mt'])

In [28]:
all_data['2025-03-20']['HD UN Equity']['mt']

{'name': 'Home Depot Inc/The',
 'figi': 'BBG000BKZB36',
 'sector': 'Consumer Discretionary'}